<a href="https://colab.research.google.com/github/abhilakshay-DataScience/Certificates/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os

# Navigate to the model folder
model_folder_path = '/content/drive/MyDrive/Saved Model'
os.chdir(model_folder_path)

In [ ]:
!pip install simple_colors

In [ ]:
from simple_colors import *

## **Class Prediction**

In [ ]:
!pip install transformers

In [ ]:
import transformers

In [ ]:
import tensorflow as tf

# Load the TensorFlow model
model_Class = tf.keras.models.load_model('textclassification.h5', custom_objects={"TFBertModel": transformers.TFBertModel})


In [ ]:
from transformers import BertTokenizer
from tqdm.auto import tqdm

In [ ]:
tokenizer_Class = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
dict={0:'business',1: 'entertainment',2: 'food',3: 'graphics',4: 'historical',5:'medical',6: 'politics',7: 'space',8: 'sport',9: 'technologie'}

In [ ]:
def prepare_data(input_text, tokenizer=tokenizer_Class):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256,
        truncation=True,
        padding='max_length',
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }




def make_prediction(processed_data,model=model_Class, classes=['business', 'entertainment', 'food', 'graphics', 'historical','medical', 'politics', 'space', 'sport', 'technologie']):
    probs = model.predict(processed_data)[0]
    return np.argmax(probs)

# **Generating Important Keywords**

In [ ]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import spacy
from spacy import displacy

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
!pip install rake_nltk && touch /root/.ipython/profile_default/startup/00-first.py && echo "import rake_nltk" >> /root/.ipython/profile_default/startup/00-first.py

In [ ]:
from rake_nltk import Rake
import nltk

In [ ]:
nltk.download('all')

In [ ]:
r = Rake(min_length=1, max_length=2,include_repeated_phrases=False,punctuations = [')','(',',',':','),',').','.'])


In [ ]:
def extractingKeywords(input_text):
  r.extract_keywords_from_text(input_text)
  r.get_ranked_phrases_with_scores()
  for rating, keyword in r.get_ranked_phrases_with_scores():
      if(rating>2):
        print(keyword.upper(),":",rating)
  return None

# **Sentiment Prediction**

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
MODEL=f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer_Sentiment=AutoTokenizer.from_pretrained(MODEL)
model_Sentiment=AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
def getKey(scores_dict,targeted_value):
  for key,value in scores_dict.items():
    if value==targeted_value:
      return key
  return None




def polarityScore(example,tokenizer=tokenizer_Sentiment,model=model_Sentiment):
  encoded_text=tokenizer(example,return_tensors="pt")
  output=model(**encoded_text)
  scores=output[0][0].detach().numpy()
  scores=softmax(scores)
  scores_dict={
    "Negative":scores[0],
    "Neutral":scores[1],
    "Position":scores[2]
  }
  i=max(scores_dict.values())
  key=getKey(scores_dict,i)

  return key


# **Text Summarizer**

In [ ]:
!pip install sumy

In [ ]:
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from termcolor import colored

In [ ]:
import textwrap
def wrap(x):
    return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [ ]:
def summary(input_text):
  summarizer  = TextRankSummarizer()
  parser = PlaintextParser.from_string(input_text,
                                      Tokenizer("english"))
  summary = summarizer(parser.document, sentences_count=4)
  for sent in summary:
      print(wrap(str(sent)),"\n")

## **Final function for all output**

In [ ]:
def class_prediction(input_text):
  processed_data = prepare_data(input_text)
  result = make_prediction(processed_data=processed_data)
  output=dict[result]
  print()
  print()
  print("This Document may belongs to "+cyan(output.upper(), 'bold')+" class.")
  print()
  print()
  print("*"*140)
  print()
  print()
  sentiment=polarityScore(input_text)
  print("The Document's data we have recieved is predicted to be : "+magenta(sentiment.upper(), 'bold'))
  print()
  print()
  print("*"*140)
  print()
  print()
  print("Important Keywords of the Document with their confidence level: ")
  print()
  extractingKeywords(input_text)
  print()
  print()
  print("*"*140)
  print()
  print()
  heading="Presenting the Summary of the Document: "
  print(blue(heading.upper(), 'bold'))
  print()
  summary(input_text)
  print()
  print()
  print("*"*140)
  print()
  print()
  print("Entities present in the Document:")
  print()
  ner_text=nlp(input_text)
  displacy.render(ner_text,style='ent',jupyter=True)


# **Taking input of Document's path**

In [ ]:
pip install pdfplumber


In [ ]:
import pdfplumber

def read_pdf(pdf_path):
    text = ""

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text()

    except Exception as e:
        print("An error occurred:", e)

    return text


In [ ]:

pdf_path=input("Enter the path for the pdf:")
text = read_pdf(pdf_path)

if text=="":
        print("No text extracted from the PDF.")


Enter the path for the pdf:/content/drive/MyDrive/sample.pdf


## **Printing output**

In [ ]:
class_prediction(text)

1/1 [==============================] - 0s 67ms/step


This Document may belongs to BUSINESS class.


********************************************************************************************************************************************


The Document's data we have recieved is predicted to be : NEUTRAL


********************************************************************************************************************************************


Important Keywords of the Document with their confidence level: 

UK ECONOMIST : 4.0
SUSTAINABLE PICK : 4.0
ROSS WALKER : 4.0
RATES ". : 4.0
RATE INCREASE : 4.0
RATE CHANGE : 4.0
RAISE RATES : 4.0
PROBABLY GOING : 4.0
ONE MEMBER : 4.0
LATEST MEETING : 4.0
KEEP RATES : 4.0
INTEREST RATES : 4.0
INFLATION FORECAST : 4.0
INCREASE MIGHT : 4.0
HOUSING MARKETS : 4.0
GROWTH RATES : 4.0
FREEZE RATES : 4.0
ENGLAND ' : 4.0
ECONOMY PANS : 4.0
ECONOMY EVOLVED : 4.0
ECONOMIC GROWTH : 4.0
DUE COURSE : 4.0
DISSENTING VOTE : 4.0
CONSUMER SPENDING : 4.0
5 %